<a href="https://colab.research.google.com/github/ayuadmy/simpasi/blob/main/SIMPASI_MODEL_SEQUENTIAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Library Needed


In [1]:
!python --version

Python 3.7.13


In [2]:
!pip install -q tensorflow-recommenders
!pip install -q tensorflowjs
!pip install -q scann

     |████████████████████████████████| 85 kB 1.4 MB/s 
     |████████████████████████████████| 77 kB 2.8 MB/s 
     |████████████████████████████████| 40 kB 5.0 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 11.2 MB 3.7 MB/s 
     |████████████████████████████████| 511.7 MB 4.9 kB/s 
     |████████████████████████████████| 1.6 MB 27.5 MB/s 
     |████████████████████████████████| 5.8 MB 7.5 MB/s 
     |████████████████████████████████| 438 kB 36.4 MB/s 


In [3]:
import pathlib
import os
import pprint
import tempfile
from google.colab import files


from typing import Dict, Text

import csv
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 

import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflowjs as tfjs
import tensorflow_recommenders as tfrs
print(tf. __version__)

2.9.1


# MPASI Menu Feedback & Resep Dataset

In [9]:
#Feedback sequential
!gdown --id 1FHCZW_U4D6L_PuO54Kv94vOuqGZ5Qgsk\

!mv "/content/feedback_sequential.csv" "/tmp"

feedback_sequential_dir = "/tmp/feedback_sequential.csv"


/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1FHCZW_U4D6L_PuO54Kv94vOuqGZ5Qgsk
To: /content/feedback_sequential.csv
100% 627/627 [00:00<00:00, 1.02MB/s]


In [10]:
#komposisi_resep_filtered
!gdown --id 1C7vDIEsSRGSlZRzz-A6Tslh0rUaig-yF\

!mv "/content/komposisi_resep_filtered.csv" "/tmp"

komposisi_resep_fix = "/tmp/komposisi_resep_filtered.csv"

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1C7vDIEsSRGSlZRzz-A6Tslh0rUaig-yF
To: /content/komposisi_resep_filtered.csv
100% 3.88k/3.88k [00:00<00:00, 4.68MB/s]


In [11]:
komposisi_resep_pd = pd.read_csv(komposisi_resep_fix)
komposisi_resep_pd.head()

,id_resep,nama_resep,nama_bahan
0,1,Agar-agar Isi Blewah,"Blewah, Melon, Agar-Agar"
1,2,Apel Rebus,Apel
2,3,Apricot Tahu,"Tahu Sutra, Pisang, Apel"
3,4,Biskuit Oat Ceri,"Unsalted Butter, Oatmeal, Ceri, Maizena"
4,5,Biskuit Oat Wortel,"Maizena, Unsalted Butter, Wortel, Oatmeal"


In [12]:
feedback_seq = pd.read_csv(feedback_sequential_dir)
feedback_seq.head()

,user_id,resep_context_id1,resep_context_id2,resep_label_id
0,1,26,54,22
1,2,67,66,63
2,3,8,19,44
3,5,36,35,69
4,6,76,74,73


In [30]:
contextid_list = []
userid_list = []
labelid_list = []

with open(feedback_sequential_dir, mode="r") as x:
  reader = csv.reader(x)
  keys = next(reader)
  for row in reader:
    userid_list.append(row[0])
    contextid_list.append([int(row[1]),int(row[2])])
    labelid_list.append(int(row[3]))


data = {keys[0]: userid_list, "resep_context_id": contextid_list, keys[3]:labelid_list }
feedback_seq_df = pd.DataFrame.from_dict(data)
print(len(feedback_seq_df))
feedback_seq_df.head()

45


,user_id,resep_context_id,resep_label_id
0,1,"[26, 54]",22
1,2,"[67, 66]",63
2,3,"[8, 19]",44
3,5,"[36, 35]",69
4,6,"[76, 74]",73


# Simulasi Aplikasi SIMPASI - User Baru

Pada user baru, customer akan melewati tahap-tahap berikut akan berupa:

1.   Log-in/Sign-up dengan email dan passsword
2.   Meminta data bayi berupa nama, tanggal lahir, bahan yang pernah dicoba sebelumnya (suka/tidak suka/alergi)
3.   Mendapatkan rekomendasi makanan dari riwayat makanan yang disukai/tidak disukai/alergi




Sehingga, sistem rekomendasi harus melakukan:


1.   Meminta username/email/password untuk masuk ke akun
2.   Meminta nama, tanggal lahir, bahan yang pernah dicoba sebelumnya *(suka/tidak suka/alergi)
3.   Menawarkan resep berdasarkan data bahan
4.   Menghitung Umur dan EER anak
5.   Merekomendasikan resep yang sesuai dengan profil
6.   Memfilter resep berdasarkan alergi bahan
7.   Memfilter menu makanan berdasarkan umur dan EER
8.   Menampilkan kombinasi menu dari resep yang cocok


## *Fixed Model of SIMPASI (FLOAT)*





In [13]:
OOV_MOVIE_ID = 0
train_examples = []
test_examples = []

def generate_examples(df):
  for item in range(len(df)):
    context = df["resep_context_id"].values[item]
    while len(context) < 2:
        context.append(OOV_MOVIE_ID)
    label = df["resep_label_id"].values[item]
    feature = {
        "resep_context_id":tf.train.Feature(float_list=tf.train.FloatList(value=context)),
        "resep_label_id":tf.train.Feature(int64_list=tf.train.Int64List(value=[label]))
    }
    tf_example = tf.train.Example(features=tf.train.Features(feature=feature))
    print(tf_example)
    if item > 30:
      test_examples.append(tf_example.SerializeToString())
    else:
      train_examples.append(tf_example.SerializeToString())
  return train_examples, test_examples


In [14]:
train_examples, test_examples = generate_examples(feedback_seq_df)

features {
  feature {
    key: "resep_context_id"
    value {
      float_list {
        value: 26.0
        value: 54.0
      }
    }
  }
  feature {
    key: "resep_label_id"
    value {
      int64_list {
        value: 22
      }
    }
  }
}

features {
  feature {
    key: "resep_context_id"
    value {
      float_list {
        value: 67.0
        value: 66.0
      }
    }
  }
  feature {
    key: "resep_label_id"
    value {
      int64_list {
        value: 63
      }
    }
  }
}

features {
  feature {
    key: "resep_context_id"
    value {
      float_list {
        value: 8.0
        value: 19.0
      }
    }
  }
  feature {
    key: "resep_label_id"
    value {
      int64_list {
        value: 44
      }
    }
  }
}

features {
  feature {
    key: "resep_context_id"
    value {
      float_list {
        value: 36.0
        value: 35.0
      }
    }
  }
  feature {
    key: "resep_label_id"
    value {
      int64_list {
        value: 69
      }
    }
  }
}

features 

In [15]:
!git clone https://github.com/tensorflow/examples
%cd examples/lite/examples/recommendation/ml/
!pip install -r requirements.txt

Cloning into 'examples'...
remote: Enumerating objects: 21552, done.
remote: Counting objects: 100% (213/213), done.
remote: Compressing objects: 100% (147/147), done.
remote: Total 21552 (delta 73), reused 153 (delta 45), pack-reused 21339
Receiving objects: 100% (21552/21552), 36.07 MiB | 10.25 MiB/s, done.
Resolving deltas: 100% (11831/11831), done.
/content/examples/lite/examples/recommendation/ml
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [16]:
!python -m data.example_generation_movielens \
  --data_dir=data/raw \
  --output_dir=data/examples \
  --min_timeline_length=3 \
  --max_context_length=10 \
  --max_context_movie_genre_length=32 \
  --min_rating=2 \
  --train_data_fraction=0.9 \
  --build_vocabs=True

2022-06-11 14:29:33.806447: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
I0611 14:29:33.807394 140362429454208 example_generation_movielens.py:460] Downloading and extracting data.
5917549/5917549 [==============================] - 2s 0us/step
I0611 14:29:37.043260 140362429454208 example_generation_movielens.py:406] Reading data to dataframes.
/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)
I0611 14:29:50.102305 140362429454208 example_generation_movielens.py:408] Generating movie rating user timelines.
I0611 14:29:59.973360 140362429454208 example_generation_movielens.py:410] Generating train and test example

In [17]:
def write_tfrecords(tf_examples, filename):
  """Write tf examples to tfrecord file."""
  with tf.io.TFRecordWriter(filename) as file_writer:
    for example in tf_examples:
      file_writer.write(example)

output_dir = '/content/examples/lite/examples/recommendation/ml/data'
OUTPUT_TRAINING_DATA_FILENAME = "train_resep_float.tfrecord"
OUTPUT_TESTING_DATA_FILENAME = "test_resep.tfrecord"

if not tf.io.gfile.exists(output_dir):
  tf.io.gfile.makedirs(output_dir)
write_tfrecords(
    tf_examples=train_examples,
    filename=os.path.join(output_dir, OUTPUT_TRAINING_DATA_FILENAME))
write_tfrecords(
    tf_examples=test_examples,
    filename=os.path.join(output_dir, OUTPUT_TESTING_DATA_FILENAME))

In [18]:
record_iterator = tf.compat.v1.io.tf_record_iterator(path="/content/examples/lite/examples/recommendation/ml/data/test_resep.tfrecord")
for string_record in record_iterator:
    example = tf.train.Example()
    example.ParseFromString(string_record)

    konteks_resep = (example.features.feature['resep_context_id'].float_list.value)
    label_resep = (example.features.feature['resep_label_id'].int64_list.value)
    print(konteks_resep, label_resep)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
[70.0, 36.0] [33]
[8.0, 12.0] [13]
[60.0, 42.0] [22]
[10.0, 49.0] [6]
[60.0, 71.0] [45]
[45.0, 17.0] [32]
[67.0, 64.0] [44]
[46.0, 59.0] [45]
[70.0, 35.0] [36]
[12.0, 41.0] [45]
[14.0, 31.0] [7]
[13.0, 42.0] [27]
[7.0, 62.0] [3]
[14.0, 48.0] [68]


From This Point, we change the output of the model from recommendation_model.py to only return ids not score (delete the top scores)

In [23]:
!python -m model.recommendation_model_launcher \
  --training_data_filepattern "data/train_resep_float.tfrecord" \
  --testing_data_filepattern "data/test_resep.tfrecord" \
  --model_dir "model/model_dir" \
  --export_dir "model/model_dir/export_m1" \
  --input_config_file "/content/SIMPASIconfig_sequential_float.pbtxt" \
  --batch_size 32 \
  --learning_rate 0.01 \
  --steps_per_epoch 2 \
  --num_epochs 10 \
  --num_eval_steps 2 \
  --run_mode "train_and_eval" \
  --gradient_clip_norm 1.0 \
  --num_predictions 10 \
  --hidden_layer_dims "32,32" \
  --eval_top_k "1,5" \
  --conv_num_filter_ratios "2,4" \
  --conv_kernel_size 4 \
  --lstm_num_units 16

2022-06-11 14:37:56.595949: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
INFO:tensorflow:Setting up train and eval input datasets.
I0611 14:37:56.597370 139874032711552 app.py:258] Setting up train and eval input datasets.
INFO:tensorflow:Build keras model for mode: train_and_eval.
I0611 14:37:56.933145 139874032711552 app.py:258] Build keras model for mode: train_and_eval.
Epoch 1/10
2/2 [==============================] - 5s 1s/step - loss: 4.5275 - Global_Recall/Recall_1: 0.0000e+00 - Global_Recall/Recall_5: 0.0156 - global_mean_rank: 43.7710 - Batch_Recall/Recall_1: 0.0312 - Batch_Recall/Recall_5: 0.1094 - batch_mean_rank: 15.0000 - val_loss: 4.4067 - val_Global_Recall/Recall_1: 0.0469 - val_Global_Recall/Recall_5: 0.0938 - val_global_mean_rank: 39.7661 - val_Batch_Recall/Recall_1: 0.0312 - val_Batch_Recall/Recall_5: 0.1875 - val_batch_mean_rank: 16.4844
Epoch 2/10
2/2 [=========================

In [24]:
!python -m model.recommendation_model_launcher \
  --training_data_filepattern "data/train_resep_float.tfrecord" \
  --testing_data_filepattern "data/test_resep.tfrecord" \
  --input_config_file "/content/SIMPASIconfig_sequential_float.pbtxt" \
  --model_dir "model/model_dir" \
  --export_dir "model/model_dir/export_m2" \
  --vocab_dir "data/examples" \
  --run_mode "export" \
  --checkpoint_path "model/model_dir/ckpt-4" \
  --num_predictions 10 \
  --hidden_layer_dims "32,32" \
  --eval_top_k "1,5" \
  --conv_num_filter_ratios "2,4" \
  --conv_kernel_size 4 \
  --lstm_num_units 16

2022-06-11 14:38:10.410373: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
INFO:tensorflow:Setting up train and eval input datasets.
I0611 14:38:10.411785 140533282686848 app.py:258] Setting up train and eval input datasets.
INFO:tensorflow:Build keras model for mode: export.
I0611 14:38:10.770158 140533282686848 app.py:258] Build keras model for mode: export.
INFO:tensorflow:Exporting model to dir: model/model_dir/export_m2
I0611 14:38:10.812489 140533282686848 app.py:258] Exporting model to dir: model/model_dir/export_m2
W0611 14:38:11.887578 140533282686848 save_impl.py:72] Skipping full serialization of Keras layer <model.recommendation_model.RecommendationModel object at 0x7fcff791b990>, because it is not built.
W0611 14:38:11.981795 140533282686848 save_impl.py:72] Skipping full serialization of Keras layer <model.label_encoder.LabelEncoder object at 0x7fcff788e9d0>, because it is not built.
W0

In [25]:
import os
import tensorflow as tf

# Use [0, 1, ... 9] as example input to represent 10 movies that user interacted with.
# Path to exported TensorFlow Lite model.
tflite_model_path = "/content/examples/lite/examples/recommendation/ml/model/model_dir/export_m2/model.tflite" #@param {type:"string"}

# Create TFLite interpreter.
interpreter = tf.lite.Interpreter(tflite_model_path)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print('Display inputs and outputs:')
print(input_details)
print(output_details)

Display inputs and outputs:
[{'name': 'serving_default_resep_context_id:0', 'index': 0, 'shape': array([2], dtype=int32), 'shape_signature': array([2], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
[{'name': 'StatefulPartitionedCall:0', 'index': 28, 'shape': array([10], dtype=int32), 'shape_signature': array([10], dtype=int32), 'dtype': <class 'numpy.int32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


In [27]:
# Find indices.
names = [
  'serving_default_resep_context_id:0',
]
indices = {i['name']: i['index'] for i in input_details}

# Fake inputs for illustration. Please change to the real data.
# Use [0, 1, ... 9] to represent 2 recipes that user interacted with.
ids = tf.range(2, dtype=float)
print(ids)
interpreter.set_tensor(indices[names[0]], ids)

# Run inference.
interpreter.invoke()

# Get outputs.
top_prediction_ids = interpreter.get_tensor(output_details[0]['index'])
print('Predicted results:')
print('Top ids: {}'.format(top_prediction_ids))

print('{} input(s):'.format(len(input_details)))
for i in range(0, len(input_details)):
    print('{} {}'.format(input_details[i]['shape'], input_details[i]['dtype']))

# Print output shape and type
print('\n{} output(s):'.format(len(output_details)))
for i in range(0, len(output_details)):
    print('{} {}'.format(output_details[i]['shape'], output_details[i]['dtype']))

tf.Tensor([0. 1.], shape=(2,), dtype=float32)
Predicted results:
Top ids: [50 24 64 17 44 53 69  7  4 21]
1 input(s):
[2] <class 'numpy.float32'>

1 output(s):
[10] <class 'numpy.int32'>


In [28]:
#List Id rekomendasi resep
id_rekomendasi = []

for i in range(0, len(top_prediction_ids)):
    id_rekomendasi.append(top_prediction_ids[i])

print(id_rekomendasi)

[50, 24, 64, 17, 44, 53, 69, 7, 4, 21]


## Corat-Coret

In [29]:
rekomendasi_df = pd.Dataframe

AttributeError: ignored

In [ ]:
df = pd.read_csv(feedback_dir)
feedback_slices = tf.data.Dataset.from_tensor_slices(dict(df))

In [ ]:
df.head()

,id_feedback,user_id,id_resep,nilai_feedback,hari,keterangan,jumlah_feedback,nama_resep,rating
0,1,1,29,1,2019-11-25,tidak_suka,2,Bubur Buah dan Sayur,2
1,2,1,71,0,2019-11-25,alergi,1,Bubur Tempe Beras Merah,3
2,3,35,138,4,2019-11-29,suka,4,Brokoli Rebus,0
3,4,35,20,1,2019-11-29,suka,1,Bubur Apricot,0
4,5,35,91,1,2019-11-29,suka,1,Yoghurt Buah Smoothie,0


In [ ]:
feedback_slices

<TensorSliceDataset element_spec={'id_feedback': TensorSpec(shape=(), dtype=tf.int64, name=None), 'user_id': TensorSpec(shape=(), dtype=tf.int64, name=None), 'id_resep': TensorSpec(shape=(), dtype=tf.int64, name=None), 'nilai_feedback': TensorSpec(shape=(), dtype=tf.int64, name=None), 'hari': TensorSpec(shape=(), dtype=tf.string, name=None), 'keterangan': TensorSpec(shape=(), dtype=tf.string, name=None), 'jumlah_feedback': TensorSpec(shape=(), dtype=tf.int64, name=None), 'nama_resep': TensorSpec(shape=(), dtype=tf.string, name=None), 'rating': TensorSpec(shape=(), dtype=tf.int64, name=None)}>

In [ ]:
feedback_slices.element_spec

{'hari': TensorSpec(shape=(), dtype=tf.string, name=None),
 'id_feedback': TensorSpec(shape=(), dtype=tf.int64, name=None),
 'id_resep': TensorSpec(shape=(), dtype=tf.int64, name=None),
 'jumlah_feedback': TensorSpec(shape=(), dtype=tf.int64, name=None),
 'keterangan': TensorSpec(shape=(), dtype=tf.string, name=None),
 'nama_resep': TensorSpec(shape=(), dtype=tf.string, name=None),
 'nilai_feedback': TensorSpec(shape=(), dtype=tf.int64, name=None),
 'rating': TensorSpec(shape=(), dtype=tf.int64, name=None),
 'user_id': TensorSpec(shape=(), dtype=tf.int64, name=None)}

In [ ]:
komposisi_resep_filtered_slices = tf.data.Dataset.from_tensor_slices(tf.cast(komposisi_resep_filtered['nama_resep'].values.reshape(-1,1), tf.string), )

In [ ]:
komposisi_resep_filtered_slices.element_spec

TensorSpec(shape=(1,), dtype=tf.string, name=None)

In [ ]:
with tf.python_io.TFRecordWriter("df.tfrecords") as writer:
    for row in csv:
        features, label = row[:-1], row[-1]
        example = tf.train.Example()
        example.features.feature["features"].float_list.value.extend(features)
        example.features.feature["label"].int64_list.value.append(label)
        writer.write(example.SerializeToString())

In [ ]:
def rename1(x0):
    y = {}
    y["nama_resep"] = x0
    return y

komposisi_resep_filtered_slices = komposisi_resep_filtered_slices.map(rename1)

In [ ]:
komposisi_resep_filtered_slices.element_spec

{'nama_resep': TensorSpec(shape=(1,), dtype=tf.string, name=None)}

In [ ]:
class SIMPASI_v2(tfrs.Model):
  def __init__(self, user_model: tf.keras.Model, resep_model: tf.keras.Model, task: tfrs.tasks.Retrieval):
      super().__init__()
      self.user_model = user_model
      self.resep_model = resep_model
      self.task = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
      user_embeddings = self.user_model(features["user_id"])
      resep_embeddings = self.resep_model(features["nama_resep"])
      return self.task(user_embeddings, resep_embeddings)

In [ ]:
rating_data = df[["user_id","nama_resep", "rating"]]

rating_data['user_id'] = rating_data.user_id.astype(np.str)
rating_data['nama_resep'] = rating_data.nama_resep.astype(np.str)
rating_data['rating'] = rating_data.rating.astype(np.float32)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launch

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((
    tf.cast(rating_data['user_id'].values.reshape(-1,1), tf.string), 
    tf.cast(rating_data['nama_resep'].values.reshape(-1,1), tf.string), 
    tf.cast(rating_data['rating'].values.reshape(-1,1),tf.float32)))

In [ ]:
dataset.element_spec

(TensorSpec(shape=(1,), dtype=tf.string, name=None),
 TensorSpec(shape=(1,), dtype=tf.string, name=None),
 TensorSpec(shape=(1,), dtype=tf.float32, name=None))

In [ ]:
def rename3(x0,x1,x2):
    y = {}
    y["user_id"] = x0
    y['nama_resep'] = x1
    y['rating'] = x2
    return y

dataset = dataset.map(rename3)

In [ ]:
dataset.element_spec

{'nama_resep': TensorSpec(shape=(1,), dtype=tf.string, name=None),
 'rating': TensorSpec(shape=(1,), dtype=tf.float32, name=None),
 'user_id': TensorSpec(shape=(1,), dtype=tf.string, name=None)}

In [ ]:
users = dataset.map(lambda x: (x["user_id"]))
resep = komposisi_resep_filtered_slices.map(lambda x: (x["nama_resep"]))

In [ ]:
print(dataset)
print(resep)
print(users)

<MapDataset element_spec={'user_id': TensorSpec(shape=(1,), dtype=tf.string, name=None), 'nama_resep': TensorSpec(shape=(1,), dtype=tf.string, name=None), 'rating': TensorSpec(shape=(1,), dtype=tf.float32, name=None)}>
<MapDataset element_spec=TensorSpec(shape=(1,), dtype=tf.string, name=None)>
<MapDataset element_spec=TensorSpec(shape=(1,), dtype=tf.string, name=None)>


In [ ]:
unique_nama_resep = np.unique(list(resep))
unique_user_ids = np.unique(list(users))

In [ ]:
print(len(unique_nama_resep))

77


In [ ]:
class RankingModel(tf.keras.Model):

  def __init__(self):
    super().__init__()
    embedding_dimension = 32

    # Compute embeddings for users.
    self.user_embeddings = tf.keras.Sequential([
      tf.keras.layers.experimental.preprocessing.StringLookup(
        vocabulary=unique_user_ids, mask_token=None),
      tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
    ])

    # Compute embeddings for books.
    self.resep_embeddings = tf.keras.Sequential([
      tf.keras.layers.experimental.preprocessing.StringLookup(
        vocabulary=unique_nama_resep, mask_token=None),
      tf.keras.layers.Embedding(len(unique_nama_resep) + 1, embedding_dimension)
    ])

    # Compute predictions.
    self.ratings = tf.keras.Sequential([
      # Learn multiple dense layers.
      tf.keras.layers.Dense(256, activation="relu"),
      tf.keras.layers.Dense(64, activation="relu"),
      # Make rating predictions in the final layer.
      tf.keras.layers.Dense(1)
  ])

  def __call__(self, x):
    
    user_id, nama_resep = x
    user_embedding = self.user_embeddings(user_id)
    resep_embedding = self.resep_embeddings(nama_resep)

    return self.ratings(tf.concat([user_embedding, resep_embedding], axis=1))

In [ ]:
RankingModel()((["42"], ["Bubur Apel"]))

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[-0.01572901]], dtype=float32)>

In [ ]:
class SIMPASIV3(tfrs.models.Model):

  def __init__(self):
    super().__init__()
    self.ranking_model: tf.keras.Model = RankingModel()
    self.task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
      loss = tf.keras.losses.MeanSquaredError(),
      metrics=[tf.keras.metrics.RootMeanSquaredError()]
    )

  def compute_loss(self, features, training=False) -> tf.Tensor:
    print(features)
    rating_predictions = self.ranking_model((features['user_id'], features["nama_resep"]))

    # The task computes the loss and the metrics.
    return self.task(labels=features["rating"], predictions=rating_predictions)

In [ ]:
from datetime import datetime

model = SIMPASIV3()
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))
# Cache the dataset 
cache_dataset = dataset.cache()
# Tensorboard 
logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
# Training 
model.fit(cache_dataset, epochs=5, verbose=1, callbacks=[tensorboard_callback])

Epoch 1/5
{'user_id': <tf.Tensor 'IteratorGetNext:2' shape=(1,) dtype=string>, 'nama_resep': <tf.Tensor 'IteratorGetNext:0' shape=(1,) dtype=string>, 'rating': <tf.Tensor 'IteratorGetNext:1' shape=(1,) dtype=float32>}
{'user_id': <tf.Tensor 'IteratorGetNext:2' shape=(1,) dtype=string>, 'nama_resep': <tf.Tensor 'IteratorGetNext:0' shape=(1,) dtype=string>, 'rating': <tf.Tensor 'IteratorGetNext:1' shape=(1,) dtype=float32>}
130/130 [==============================] - 3s 5ms/step - root_mean_squared_error: 2.0721 - loss: 4.2839 - regularization_loss: 0.0000e+00 - total_loss: 4.2839
Epoch 2/5
130/130 [==============================] - 1s 10ms/step - root_mean_squared_error: 1.6231 - loss: 2.6161 - regularization_loss: 0.0000e+00 - total_loss: 2.6161
Epoch 3/5
130/130 [==============================] - 1s 4ms/step - root_mean_squared_error: 1.4452 - loss: 2.1171 - regularization_loss: 0.0000e+00 - total_loss: 2.1171
Epoch 4/5
130/130 [==============================] - 1s 5ms/step - root_mean

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logdir

<IPython.core.display.Javascript object>

In [ ]:
!kill 762

/bin/bash: line 0: kill: (762) - No such process


In [ ]:
# Create array with users id in every place on the lenght of the unique books number
user8 = np.array(["8" for i in range(len(unique_nama_resep))])

# Convert it to tf.data.Dataset 
test_data = tf.data.Dataset.from_tensor_slices((tf.cast(user8.reshape(-1,1), tf.string), tf.cast(unique_nama_resep.reshape(-1,1), tf.string)))

# Name the columns 
@tf.function
def rename2(x0,x1):
    y = {}
    y["user_id"] = x0
    y['nama_resep'] = x1
    return y
test_data = test_data.map(rename2)

# Now lets make predictions and store them in to dictionary
test_ratings = {}

for b in test_data:
    test_ratings[b['nama_resep'].numpy()[0]] = model.ranking_model((b['user_id'],b['nama_resep']))
    print(b)

# sort them by score and print the titles
for b in sorted(test_ratings, key=test_ratings.get, reverse=True):
    print(b)

{'user_id': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'8'], dtype=object)>, 'nama_resep': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'Agar-agar Isi Blewah'], dtype=object)>}
{'user_id': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'8'], dtype=object)>, 'nama_resep': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'Apel Rebus'], dtype=object)>}
{'user_id': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'8'], dtype=object)>, 'nama_resep': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'Apricot Tahu'], dtype=object)>}
{'user_id': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'8'], dtype=object)>, 'nama_resep': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'Biskuit Oat Ceri'], dtype=object)>}
{'user_id': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'8'], dtype=object)>, 'nama_resep': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'Biskuit Oat Wortel'], dtype=object)>}
{'user_id': <tf.Tensor: shape=(1,), dtype=string, numpy=arr

In [ ]:
nama_resep = [a for a in df["nama_resep"]]
user_id = [str(a) for a in df["user_id"]]
rating = [str(a) for a in df["rating"]]

print(nama_resep)
print(user_id)
print(rating)

['Bubur Buah dan Sayur', 'Bubur Tempe Beras Merah', 'Brokoli Rebus', 'Bubur Apricot', 'Yoghurt Buah Smoothie', 'Bubur Ayam Talas Keju', 'Bubur Pisang Oat', 'Bubur Kentang Tomat', 'Brokoli Rebus', 'Brokoli Keju', 'Bubur Apricot', 'Bubur Pisang', 'Bubur Buah dan Sayur', 'Bubur Pisang Jeruk', 'Brokoli Rebus', 'Brokoli Keju', 'Bubur Apricot', 'Bubur Wortel', 'Bubur Ceri Alpukat', 'Bubur Tomat Kacang Merah', 'Brokoli Keju', 'Bubur Hunkwe Susu Keju', 'Bubur Pisang Tomat', 'Bola Ikan', 'Pure Tepung Beras Putih, Zukini, dan Ubi Kuning', 'Pure Apel dan Papaya', 'Bubur Ubi Kembang Kol', 'Bubur Pisang Tomat', 'Bubur Beras Merah', 'Oatmeal Kurma', 'Puding Jeruk Pepaya', 'Bubur Ubi Kembang Kol', 'Pure Tepung Beras Putih, Zukini, dan Ubi Kuning\r\n', 'Bubur Plum', 'Bubur Makaroni Teri Bayam', 'Perkedel Lele', 'Lapis Mangga', 'Bubur Tahu', 'Daging Ungkep', 'Perkedel Lele', 'Bubur Ceri Alpukat', 'Bubur Pisang Jeruk', 'Bubur Kentang Edamame', 'Bubur Kentang', 'Bubur Pisang Jeruk', 'Bubur Wortel', 'Bubu

In [ ]:
zippedList = [{"nama_resep": name, "user_id": id, "resep_rating": stars} for name, id, stars in zip(nama_resep, user_id, rating)]

dataset = tf.data.Dataset.from_generator(
lambda: zippedList, {"nama_resep": tf.string, "user_id": tf.string, "resep_rating": tf.string},
output_shapes={"nama_resep": tf.shape(0), "user_id": tf.shape(0), "resep_rating": tf.shape(0)}
)

print(dataset)

<FlatMapDataset element_spec={'nama_resep': TensorSpec(shape=(), dtype=tf.string, name=None), 'user_id': TensorSpec(shape=(), dtype=tf.string, name=None), 'resep_rating': TensorSpec(shape=(), dtype=tf.string, name=None)}>


In [ ]:
ratings = dataset.map(
lambda x: {"nama_resep": x["nama_resep"], "user_id":x["user_id"]}
)

user_ids_vocabulary = tf.keras.layers.experimental.preprocessing.StringLookup(mask_token=None)

user_ids_vocabulary.adapt(dataset.map(lambda x: x["user_id"]))

nama_resep_vocabulary = tf.keras.layers.experimental.preprocessing.StringLookup(mask_token=None)

nama_resep_vocabulary.adapt(dataset.map(lambda x: x["nama_resep"]))

nama_resep = dataset.map(lambda x: x["nama_resep"])

user_model = tf.keras.Sequential([user_ids_vocabulary, tf.keras.layers.Embedding(user_ids_vocabulary.vocab_size(), 64)])
                                 
resep_model = tf.keras.Sequential([nama_resep_vocabulary, tf.keras.layers.Embedding(nama_resep_vocabulary.vocab_size(), 64)])

task = tfrs.tasks.Retrieval(
metrics=tfrs.metrics.FactorizedTopK(nama_resep.batch(4).map(resep_model)))

model = SIMPASI_v2(user_model, resep_model, task)

model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))

model.fit(ratings.batch(16), epochs=8)

Epoch 1/8
9/9 [==============================] - 3s 180ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0077 - factorized_top_k/top_10_categorical_accuracy: 0.0231 - factorized_top_k/top_50_categorical_accuracy: 0.3538 - factorized_top_k/top_100_categorical_accuracy: 0.8846 - loss: 35.7427 - regularization_loss: 0.0000e+00 - total_loss: 35.7427
Epoch 2/8
9/9 [==============================] - 1s 124ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0154 - factorized_top_k/top_5_categorical_accuracy: 0.1000 - factorized_top_k/top_10_categorical_accuracy: 0.2077 - factorized_top_k/top_50_categorical_accuracy: 0.9000 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 30.7642 - regularization_loss: 0.0000e+00 - total_loss: 30.7642
Epoch 3/8
9/9 [==============================] - 1s 117ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0154 - factorized_top_k/top_5_categorical_accuracy: 0.1846 - facto

In [ ]:
# Create a model that takes in raw query features, and
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
# recommends movies out of the entire movies dataset.
index.index_from_dataset(
  tf.data.Dataset.zip((nama_resep.batch(100), nama_resep.batch(100).map(model.resep_model)))
)

# Get recommendations.
_, titles = index(tf.constant(["10"]))
print(f"Recommendations for user 10: {titles}")

Recommendations for user 10: [[b'Bubur Plum' b'Bubur Plum' b'Bubur Plum' b'Bubur Kembang Kol'
  b'Bubur Ubi Kembang Kol' b'Bubur Ubi Kembang Kol'
  b'Bubur Ubi Kembang Kol' b'Bubur Ubi Kembang Kol'
  b'Pure Tepung Beras Putih, Zukini, dan Ubi Kuning'
  b'Pure Tepung Beras Putih, Zukini, dan Ubi Kuning']]


In [ ]:
with tempfile.TemporaryDirectory() as tmp:

  # Save the index.
  tf.saved_model.save(obj=index, export_dir="export")

converter = tf.lite.TFLiteConverter.from_saved_model("export")
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: export/assets


INFO:tensorflow:Assets written to: export/assets


In [ ]:
tflite_model_file = pathlib.Path("/content/converted_model.tflite")
tflite_model_file.write_bytes(tflite_model)
files.download(tflite_model_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

In [ ]:
feedback = feedback_slices.map(lambda x: {
    "nama_resep": x["nama_resep"],
    "user_id": tf.strings.as_string(x["user_id"]),
})
menu = komposisi_resep_filtered_slices.map(lambda x: x["nama_resep"])

In [ ]:
tf.random.set_seed(12)
shuffled = feedback.shuffle(100, seed=12, reshuffle_each_iteration=False)

train = shuffled.take(80)
test = shuffled.skip(80).take(20)

In [ ]:
nama_resep = menu.batch(10)
user_ids = feedback.batch(100).map(lambda x: (x["user_id"]))

unique_menu = np.unique(np.concatenate(list(nama_resep)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

In [ ]:
print(nama_resep.element_spec)
print(user_ids.element_spec)

TensorSpec(shape=(None,), dtype=tf.string, name=None)
TensorSpec(shape=(None,), dtype=tf.string, name=None)


In [ ]:
print(len(unique_menu))
print(unique_menu)
print(len(unique_user_ids))
print(unique_user_ids)

77
[b'Agar-agar Isi Blewah' b'Apel Rebus' b'Apricot Tahu' b'Biskuit Oat Ceri'
 b'Biskuit Oat Wortel' b'Biskuit Sereal' b'Bola-bola Ubi' b'Bubur Apel'
 b'Bubur Ayam Kedelai' b'Bubur Ayam Talas Keju' b'Bubur Beras Merah'
 b'Bubur Buah dan Sayur' b'Bubur Ceri Alpukat' b'Bubur Hunkwe Susu Keju'
 b'Bubur Jagung Bayam Tahu' b'Bubur Jagung Susu' b'Bubur Jagung Wortel'
 b'Bubur Kacang Hijau Kurma' b'Bubur Kacang Polong' b'Bubur Kembang Kol'
 b'Bubur Kentang' b'Bubur Kentang Edamame' b'Bubur Labu Brokoli'
 b'Bubur Makaroni Teri Bayam' b'Bubur Oatmeal' b'Bubur Pepaya'
 b'Bubur Pir' b'Bubur Pisang' b'Bubur Pisang Jeruk'
 b'Bubur Pisang Kentang' b'Bubur Pisang Tomat' b'Bubur Pisang Ubi'
 b'Bubur Plum' b'Bubur Sawi Wortel' b'Bubur Singkong Apel'
 b'Bubur Susu Kacang Polong' b'Bubur Tahu' b'Bubur Tahu Wortel'
 b'Bubur Talas Apel' b'Bubur Tempe' b'Bubur Tempe Beras Merah'
 b'Bubur Tomat Kacang Merah' b'Bubur Ubi Bayam Kacang Polong'
 b'Bubur Ubi Kembang Kol' b'Bubur Wortel' b'Daging Ungkep'
 b'Ikan K

In [ ]:
embedding_dimension = 16

In [ ]:
#Query Tower
user_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_user_ids, mask_token=None),
  # We add an additional embedding to account for unknown tokens.
  tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
])

In [ ]:
#Candidate Tower
menu_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_menu, mask_token=None),
  tf.keras.layers.Embedding(len(unique_menu) + 1, embedding_dimension)
])

In [ ]:
#metrics
metrics = tfrs.metrics.FactorizedTopK(
  candidates=menu.batch(64).map(menu_model)
)

In [ ]:
#loss
task = tfrs.tasks.Retrieval(
  metrics=metrics
)

In [ ]:
class SIMPASIModel(tfrs.Model):

  def __init__(self, user_model, menu_model):
    super().__init__()
    self.menu_model: tf.keras.Model = menu_model
    self.user_model: tf.keras.Model = user_model
    self.task: tf.keras.layers.Layer = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["user_id"])
    # And pick out the movie features and pass them into the movie model,
    # getting embeddings back.
    positive_menu_embeddings = self.menu_model(features["nama_resep"])

    # The task computes the loss and the metrics.
    return self.task(user_embeddings, positive_menu_embeddings)

In [ ]:
class NoBaseClassSIMPASIModel(tf.keras.Model):

  def __init__(self, user_model, menu_model):
    super().__init__()
    self.menu_model: tf.keras.Model = menu_model
    self.user_model: tf.keras.Model = user_model
    self.task: tf.keras.layers.Layer = task

  def train_step(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:

    # Set up a gradient tape to record gradients.
    with tf.GradientTape() as tape:

      # Loss computation.
      user_embeddings = self.user_model(features["user_id"])
      positive_menu_embeddings = self.menu_model(features["nama_resep"])
      loss = self.task(user_embeddings, positive_menu_embeddings)

      # Handle regularization losses as well.
      regularization_loss = sum(self.losses)

      total_loss = loss + regularization_loss

    gradients = tape.gradient(total_loss, self.trainable_variables)
    self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

    metrics = {metric.name: metric.result() for metric in self.metrics}
    metrics["loss"] = loss
    metrics["regularization_loss"] = regularization_loss
    metrics["total_loss"] = total_loss

    return metrics

  def test_step(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:

    # Loss computation.
    user_embeddings = self.user_model(features["user_id"])
    positive_menu_embeddings = self.movie_model(features["nama_resep"])
    loss = self.task(user_embeddings, positive_menu_embeddings)

    # Handle regularization losses as well.
    regularization_loss = sum(self.losses)

    total_loss = loss + regularization_loss

    metrics = {metric.name: metric.result() for metric in self.metrics}
    metrics["loss"] = loss
    metrics["regularization_loss"] = regularization_loss
    metrics["total_loss"] = total_loss

    return metrics

In [ ]:
model = SIMPASIModel(user_model, menu_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [ ]:
cached_train = train.shuffle(10).batch(16).cache()
cached_test = test.batch(8).cache()

In [ ]:
cached_train.element_spec

{'nama_resep': TensorSpec(shape=(None,), dtype=tf.string, name=None),
 'user_id': TensorSpec(shape=(None,), dtype=tf.string, name=None)}

In [ ]:
print(cached_test)

<CacheDataset element_spec={'nama_resep': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'user_id': TensorSpec(shape=(None,), dtype=tf.string, name=None)}>


In [ ]:
history = model.fit(cached_train, epochs=10)

Epoch 1/10
5/5 [==============================] - 2s 50ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0625 - factorized_top_k/top_10_categorical_accuracy: 0.1625 - factorized_top_k/top_50_categorical_accuracy: 0.7250 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 44.3568 - regularization_loss: 0.0000e+00 - total_loss: 44.3568
Epoch 2/10
5/5 [==============================] - 0s 53ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0625 - factorized_top_k/top_5_categorical_accuracy: 0.4750 - factorized_top_k/top_10_categorical_accuracy: 0.6875 - factorized_top_k/top_50_categorical_accuracy: 0.9750 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 44.2037 - regularization_loss: 0.0000e+00 - total_loss: 44.2037
Epoch 3/10
5/5 [==============================] - 0s 46ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0750 - factorized_top_k/top_5_categorical_accuracy: 0.8375 - facto

In [ ]:
model.evaluate(cached_test, return_dict=True)

3/3 [==============================] - 0s 53ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.1500 - factorized_top_k/top_10_categorical_accuracy: 0.2000 - factorized_top_k/top_50_categorical_accuracy: 0.6500 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 11.4629 - regularization_loss: 0.0000e+00 - total_loss: 11.4629


{'factorized_top_k/top_100_categorical_accuracy': 1.0,
 'factorized_top_k/top_10_categorical_accuracy': 0.20000000298023224,
 'factorized_top_k/top_1_categorical_accuracy': 0.0,
 'factorized_top_k/top_50_categorical_accuracy': 0.6499999761581421,
 'factorized_top_k/top_5_categorical_accuracy': 0.15000000596046448,
 'loss': 6.065029144287109,
 'regularization_loss': 0,
 'total_loss': 6.065029144287109}

In [ ]:
# Create a model that takes in raw query features, and
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
# recommends movies out of the entire movies dataset.
index.index_from_dataset(
  tf.data.Dataset.zip((menu.batch(100), menu.batch(100).map(model.menu_model)))
)

# Get recommendations.
_, titles = index(tf.constant(["10"]))
print(f"Recommendations for user 10: {titles}")

Recommendations for user 10: [[b'Bubur Kentang Edamame' b'Pure Ubi Kuning dan Beras Merah'
  b'Bubur Susu Kacang Polong' b'Bubur Singkong Apel' b'Pure Tiga Rasa'
  b'Bubur Oatmeal' b'Perkedel Lele' b'Bubur Wortel' b'Daging Ungkep'
  b'Bola-bola Ubi']]


In [ ]:
# Export the query model.
with tempfile.TemporaryDirectory() as tmp:

  # Save the index.
  tf.saved_model.save(obj=index, export_dir="export")

  # Load it back; can also be done in TensorFlow Serving.
  loaded = tf.saved_model.load("export")

  print(loaded)
  # Pass a user id in, get top predicted movie titles back.
  scores, titles = loaded(["32"])

  print(f"Recommendations: {titles[0][:3]}")
  loaded

INFO:tensorflow:Assets written to: export/assets


INFO:tensorflow:Assets written to: export/assets


<tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject object at 0x7ff0db0fa790>
Recommendations: [b'Bubur Talas Apel' b'Bubur Jagung Wortel' b'Kukis Havermut Apel']


In [ ]:
#tflite
converter = tf.lite.TFLiteConverter.from_saved_model("export")
tflite_model = converter.convert()
open("converted_model.tflite", "wb").write(tflite_model)

12392

In [ ]:
#h5
model.save_weights('model_weight', save_format='tf')

In [ ]:
pb_model_dir = "/content/saved_model.pb"
h5_model = "/mymodel.h5"

In [ ]:
# Loading the Tensorflow Saved Model (PB)
model_v2 = tf.keras.models.load_model(pb_model_dir)
print(model.summary())

# Saving the Model in H5 Format
tf.keras.models.save_model(model_v2, h5_model)

# Loading the H5 Saved Model
loaded_model_from_h5 = tf.keras.models.load_model(h5_model)
print(loaded_model_from_h5.summary())

OSError: ignored

In [ ]:
model = tf.keras.models.load_model(pb_model_dir)
print(model.summary())

NameError: ignored

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

ValueError: ignored

In [ ]:
tflite_model_file = pathlib.Path("/content/converted_model.tflite")
tflite_model_file.write_bytes(tflite_model)

12392

In [ ]:
files.download(tflite_model_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#Tensorflow JS
# import time
# saved_model_path = "tmp/saved_models/{}".format(int(time.time()))

# tf.keras.experimental.export_saved_model(model,saved_model_path)

model.save("/content/")

ValueError: ignored

In [ ]:
# tensorflowjs_converter --input_format=keras /content/model_weight /tmp/tfjs_model
tensorflowjs_converter \
    --input_format=tf_saved_model \
    /content/model_weight \
    /tmp/tfjs_model
# tfjs.converters.save_keras_model(model_weight,"model_weights")

In [ ]:
interpreter = tf.lite.Interpreter(model_path="converted_model.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print(input_details)
print(output_details)

print(input_details[0])
# Test the model.
interpreter.set_tensor(input_details[0]["index"], np.array(["32"]))

interpreter.invoke()

reccomendation = interpreter.get_tensor(output_details[1]['index'])
rating = interpreter.get_tensor(output_details[0]['index'])
print(reccomendation)
print(rating)

# Print input shape and type
print('{} input(s):'.format(len(input_details)))
for i in range(0, len(input_details)):
    print('{} {}'.format(input_details[i]['shape'], input_details[i]['dtype']))

# Print output shape and type
print('\n{} output(s):'.format(len(output_details)))
for i in range(0, len(output_details)):
    print('{} {}'.format(output_details[i]['shape'], output_details[i]['dtype']))

[{'name': 'serving_default_input_1:0', 'index': 0, 'shape': array([1], dtype=int32), 'shape_signature': array([-1], dtype=int32), 'dtype': <class 'numpy.bytes_'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
[{'name': 'StatefulPartitionedCall_1:0', 'index': 10, 'shape': array([ 1, 10], dtype=int32), 'shape_signature': array([-1, 10], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}, {'name': 'StatefulPartitionedCall_1:1', 'index': 12, 'shape': array([ 1, 10], dtype=int32), 'shape_signature': array([-1, 10], dtype=int32), 'dtype': <class 'numpy.bytes_'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtyp

In [ ]:
rekomendasi = 

In [ ]:
titles_numpy = np.array(titles).transpose().astype(str)

rekomendasi = pd.DataFrame(titles_numpy, columns=["rekomendasi_resep"])
rekomendasi

,rekomendasi_resep
0,Bubur Kentang Edamame
1,Pure Ubi Kuning dan Beras Merah
2,Bubur Susu Kacang Polong
3,Bubur Singkong Apel
4,Pure Tiga Rasa
5,Bubur Oatmeal
6,Perkedel Lele
7,Bubur Wortel
8,Daging Ungkep
9,Bola-bola Ubi


*italicized text*## *Sistem Rekomendasi Menu MPASI*

In [8]:
contextid_list = []
userid_list = []
labelid_list = []

with open(feedback_sequential_dir, mode="r") as x:
  reader = csv.reader(x)
  keys = next(reader)
  for row in reader:
    userid_list.append(row[0])
    contextid_list.append([int(row[1]),int(row[2])])
    labelid_list.append(int(row[3]))


data = {keys[0]: userid_list, "resep_context_id": contextid_list, keys[3]:labelid_list }
feedback_seq_df = pd.DataFrame.from_dict(data)
print(len(feedback_seq_df))
feedback_seq_df.head()

45


,user_id,resep_context_id,resep_label_id
0,1,"[26, 54]",22
1,2,"[67, 66]",63
2,3,"[8, 19]",44
3,5,"[36, 35]",69
4,6,"[76, 74]",73


In [ ]:
OOV_MOVIE_ID = 0
train_examples = []
test_examples = []

def generate_examples(df):
  for item in range(len(df)):
    context = df["resep_context_id"].values[item]
    while len(context) < 2:
        context.append(OOV_MOVIE_ID)
    label = df["resep_label_id"].values[item]
    feature = {
        "resep_context_id":tf.train.Feature(int64_list=tf.train.Int64List(value=context)),
        "resep_label_id":tf.train.Feature(int64_list=tf.train.Int64List(value=[label]))
    }
    tf_example = tf.train.Example(features=tf.train.Features(feature=feature))
    print(tf_example)
    if item > 30:
      test_examples.append(tf_example.SerializeToString())
    else:
      train_examples.append(tf_example.SerializeToString())
  return train_examples, test_examples


In [ ]:
train_examples, test_examples = generate_examples(feedback_seq_df)

features {
  feature {
    key: "resep_context_id"
    value {
      int64_list {
        value: 26
        value: 54
      }
    }
  }
  feature {
    key: "resep_label_id"
    value {
      int64_list {
        value: 22
      }
    }
  }
}

features {
  feature {
    key: "resep_context_id"
    value {
      int64_list {
        value: 67
        value: 66
      }
    }
  }
  feature {
    key: "resep_label_id"
    value {
      int64_list {
        value: 63
      }
    }
  }
}

features {
  feature {
    key: "resep_context_id"
    value {
      int64_list {
        value: 8
        value: 19
      }
    }
  }
  feature {
    key: "resep_label_id"
    value {
      int64_list {
        value: 44
      }
    }
  }
}

features {
  feature {
    key: "resep_context_id"
    value {
      int64_list {
        value: 36
        value: 35
      }
    }
  }
  feature {
    key: "resep_label_id"
    value {
      int64_list {
        value: 69
      }
    }
  }
}

features {
  feature {
  

In [ ]:
!git clone https://github.com/tensorflow/examples
%cd examples/lite/examples/recommendation/ml/
!pip install -r requirements.txt

Cloning into 'examples'...
remote: Enumerating objects: 21552, done.
remote: Counting objects: 100% (213/213), done.
remote: Compressing objects: 100% (148/148), done.
remote: Total 21552 (delta 73), reused 154 (delta 44), pack-reused 21339
Receiving objects: 100% (21552/21552), 36.06 MiB | 26.06 MiB/s, done.
Resolving deltas: 100% (11834/11834), done.
/content/examples/lite/examples/recommendation/ml
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!python -m data.example_generation_movielens \
  --data_dir=data/raw \
  --output_dir=data/examples \
  --min_timeline_length=3 \
  --max_context_length=10 \
  --max_context_movie_genre_length=32 \
  --min_rating=2 \
  --train_data_fraction=0.9 \
  --build_vocabs=True

2022-06-10 03:09:52.920084: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
I0610 03:09:52.921065 140066213521280 example_generation_movielens.py:460] Downloading and extracting data.
5917549/5917549 [==============================] - 0s 0us/step
I0610 03:09:54.077274 140066213521280 example_generation_movielens.py:406] Reading data to dataframes.
/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)
I0610 03:10:13.296033 140066213521280 example_generation_movielens.py:408] Generating movie rating user timelines.
I0610 03:10:20.746802 140066213521280 example_generation_movielens.py:410] Generating train and test example

In [ ]:
def write_tfrecords(tf_examples, filename):
  """Write tf examples to tfrecord file."""
  with tf.io.TFRecordWriter(filename) as file_writer:
    for example in tf_examples:
      file_writer.write(example)

output_dir = '/content/examples/lite/examples/recommendation/ml/data'
OUTPUT_TRAINING_DATA_FILENAME = "train_resep.tfrecord"
OUTPUT_TESTING_DATA_FILENAME = "test_resep.tfrecord"

if not tf.io.gfile.exists(output_dir):
  tf.io.gfile.makedirs(output_dir)
write_tfrecords(
    tf_examples=train_examples,
    filename=os.path.join(output_dir, OUTPUT_TRAINING_DATA_FILENAME))
write_tfrecords(
    tf_examples=test_examples,
    filename=os.path.join(output_dir, OUTPUT_TESTING_DATA_FILENAME))

In [ ]:
record_iterator = tf.compat.v1.io.tf_record_iterator(path="/content/examples/lite/examples/recommendation/ml/data/test_resep.tfrecord")
for string_record in record_iterator:
    example = tf.train.Example()
    example.ParseFromString(string_record)

    i = (example.features.feature['resep_context_id'].int64_list.value)
    data = (example.features.feature['resep_label_id'].int64_list.value)
    print(i, data)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
[70, 36] [33]
[8, 12] [13]
[60, 42] [22]
[10, 49] [6]
[60, 71] [45]
[45, 17] [32]
[67, 64] [44]
[46, 59] [45]
[70, 35] [36]
[12, 41] [45]
[14, 31] [7]
[13, 42] [27]
[7, 62] [3]
[14, 48] [68]


In [ ]:
!python -m model.recommendation_model_launcher \
  --training_data_filepattern "data/train_resep.tfrecord" \
  --testing_data_filepattern "data/test_resep.tfrecord" \
  --model_dir "model/model_dir" \
  --export_dir "model/model_dir/export_m1" \
  --input_config_file "/content/SIMPASIconfig.pbtxt" \
  --batch_size 32 \
  --learning_rate 0.01 \
  --steps_per_epoch 2 \
  --num_epochs 2 \
  --num_eval_steps 2 \
  --run_mode "train_and_eval" \
  --gradient_clip_norm 1.0 \
  --num_predictions 10 \
  --hidden_layer_dims "32,32" \
  --eval_top_k "1,5" \
  --conv_num_filter_ratios "2,4" \
  --conv_kernel_size 4 \
  --lstm_num_units 16

In [ ]:
import os
import tensorflow as tf

# Use [0, 1, ... 9] as example input to represent 10 movies that user interacted with.
context = tf.range(10)
# Path to exported TensorFlow Lite model.
tflite_model_path = "/content/examples/lite/examples/recommendation/ml/model/model_dir/export_m1/model.tflite" #@param {type:"string"}

# Create TFLite interpreter.
interpreter = tf.lite.Interpreter(tflite_model_path)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print('Display inputs and outputs:')
print(input_details)
print(output_details)

In [ ]:
# Find indices.
names = [
  'serving_default_resep_context_id:0',
]
indices = {i['name']: i['index'] for i in input_details}

# Fake inputs for illustration. Please change to the real data.
# Use [0, 1, ... 9] to represent 2 recipes that user interacted with.
ids = tf.range(2)
print(ids)
interpreter.set_tensor(indices[names[0]], ids)

# Run inference.
interpreter.invoke()

# Get outputs.
top_prediction_ids = interpreter.get_tensor(output_details[1]['index'])
top_prediction_scores = interpreter.get_tensor(output_details[0]['index'])
print('Predicted results:')
print('Top ids: {}'.format(top_prediction_ids))
print('Top scores: {}'.format(top_prediction_scores))

print('{} input(s):'.format(len(input_details)))
for i in range(0, len(input_details)):
    print('{} {}'.format(input_details[i]['shape'], input_details[i]['dtype']))

# Print output shape and type
print('\n{} output(s):'.format(len(output_details)))
for i in range(0, len(output_details)):
    print('{} {}'.format(output_details[i]['shape'], output_details[i]['dtype']))

In [ ]:
#List Id rekomendasi resep
id_rekomendasi = []

for i in range(0, len(top_prediction_ids)):
    id_rekomendasi.append(top_prediction_ids[i])

print(id_rekomendasi)

In [ ]:
#List Nama Rekomendasi Resep
nama_rekomendasi= []

for x in rekomendasi:
  index_x = komposisi_resep_pd.index[komposisi_resep_pd["id_resep"] == x].tolist()
  result = komposisi_resep_pd.loc[index_x, ["nama_resep"]]
  nama_rekomendasi.append(result.nama_resep.values[0])

print(nama_rekomendasi)

In [ ]:
#Filter Hasil Rekomendasi Berdasar Alergi dan Tidak Suka
alergi = ["Kerang"]
tidak_suka = []
id_rekomendasi_nonalergen = []
nama_rekomendasi_nonalergen = []

for x in nama_rekomendasi:
  index_x = komposisi_resep_pd.index[komposisi_resep_pd["nama_resep"] == x].tolist()
  result = komposisi_resep_pd.loc[index_x, ["nama_bahan"]]
  mask = result.nama_bahan.str.contains(search_list(alergi))
  if ~mask.any():
    id_rekomendasi_nonalergen.append(int(index_x[0]))
    nama_rekomendasi_nonalergen.append(x)

for x in nama_rekomendasi_nonalergen:
  index_x = komposisi_resep_pd.index[komposisi_resep_pd["nama_resep"] == x].tolist()
  result = komposisi_resep_pd.loc[index_x, ["nama_bahan"]]
  mask = result.nama_bahan.str.contains(search_list(tidak_suka))
  if ~mask.any():
    id_rekomendasi_nonalergen.append(int(index_x[0]))
    nama_rekomendasi_nonalergen.append(x)

print(id_rekomendasi_nonalergen)
print(nama_rekomendasi_nonalergen)